# What this does

## Goal
- We want to find vocabularies that is commonly used in fakenews newspapers 

## How
1. Create a list of bigrams
2. Count their occurrences in the mainstream articles dataset
3. Count their occurrences in the fakenews articles dataset
4. Compare the differences of appreance of bigrams in mainstream articles and fakenews articles

## Results
- Identified bigrams commonly used in fakenews articles and not used in mainstream articles.
- A list of bigrams commonly used in fakenees with their frequency.

In [1]:
import sys
sys.path.append('../src/analyse')
from counter import Counter
from collections import Counter as CollCount
import nltk
from tqdm import tqdm

In [2]:
import numpy as np
counters = np.load('../data/articles_sample.npy', allow_pickle=True)

In [3]:
def find(counter, words, bigrams, min_appearences=0, max_appearances=9223372036854775807):
    if words:
        fdist = counter.get_words()
    elif bigrams:
        fdist = counter.get_bigrams()
    else:
        return
    keys = set()
    for k,v in fdist.items():
        if min_appearences < v < max_appearances:
            keys.add(k)
    return keys , fdist

def shared(counter_a, counter_b, words, bigrams, appearences):
    """ outputs a set of words that appear at least t times in both a and b"""
    keys_a, fdist = find(counter_a, words, bigrams, appearences)
    keys_b, fdist = find(counter_b,words, bigrams, appearences)
    return keys_a & keys_b

def shared_multiple_sets(counters, words, bigrams, nr_sites, real, min_appearences=0, max_appearances=1000):
    """ Outputs words that appear at least t times and at least in x sets with the same label for real"""
    all = set()
    all_sets = []
    final_set = set()
    count_freq = nltk.FreqDist()
    for counter in tqdm(counters):
        # Do note, real here is a boolean, so can be false, it checks if "real" are the same
        print(counter)
        if counter.real==real:
            new_set,new_freq = find(counter, words, bigrams, min_appearences=min_appearences, max_appearances=max_appearances)
            all_sets.append(new_set)
            all = all|new_set
            count_freq = count_freq+new_freq
    for word in all:
        count = 0
        for i in all_sets:
            if word in i:
                count+=1
        if count>=nr_sites:
            final_set.add(word)
    return final_set, count_freq


def difference(counters, words, bigrams, nr_fake_sites,nr_real_sites,fake_appearences, real_appearances):
    """ Outputs words which appear in at least nr_fakes_sites fake sites at least min_appearances times, but not in nr_real_sites real (below threshold)"""
    frequent_fake_words, fake_count_freq = shared_multiple_sets(counters,words, bigrams, nr_fake_sites,False,min_appearences=fake_appearences,max_appearances=999999)
    frequent_real_words, real_count_freq = shared_multiple_sets(counters,words, bigrams, nr_real_sites,True, min_appearences=real_appearances,max_appearances=999999)
    return frequent_fake_words - frequent_real_words, fake_count_freq, real_count_freq

In [4]:
""" Below prints differences for words"""
def different_words(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1):
    words, fake_freq, real_freq = difference(counters, words = True, bigrams=False,nr_fake_sites = nr_fake_sites, nr_real_sites=nr_real_sites,fake_appearences=fake_appearences, real_appearances=real_appearances)
    total_fake = sum(fake_freq.values())
    total_real = sum(real_freq.values())
    print(total_fake,", ", total_real)
    for word in words:
        # print(word, ", fake count:", fake_freq.get(word), ",real count:", real_freq.get(word))
        print(word, ",", fake_freq.get(word), ",", real_freq.get(word))

def different_bigrams(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1):
    bigrams, fake_freq, real_freq = difference(counters, words = False, bigrams=True,nr_fake_sites = nr_fake_sites, nr_real_sites=nr_real_sites,fake_appearences=fake_appearences, real_appearances=real_appearances)
    for bi in bigrams:
        # print(bi, ", fake count:", fake_freq.get(bi), ",real count:", real_freq.get(bi))
        print(bi, ",", fake_freq.get(bi), ",", real_freq.get(bi))

def different_bigrams_weight(counters):
    """ This function outputs the bigrams and their weight over all bigrams"""
    bigrams, fake_freq, real_freq = difference(counters, words = False, bigrams=True,nr_fake_sites = 1, nr_real_sites=1,fake_appearences=0, real_appearances=10)
    total_fake = sum(fake_freq.values())
    total_real = sum(real_freq.values())
    print(total_fake,", ", total_real)
    for bi in bigrams:
        # prints bigram, frequency in fake, weight of total, frequeuncy of real, weight of total bigrams
        print(bi, ",", fake_freq.get(bi), ",", real_freq.get(bi))

In [5]:
different_words(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.25it/s]

66877 ,  757714
this , 160 , 1
that , 321 , 1
adhd , 117 , 1
measles , 124 , None
were , 104 , None
statines , 105 , None
cholesterol , 121 , 1


In [6]:
different_bigrams(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=25, real_appearances=50)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:03<00:00,  1.93s/it]

('whooping', 'cough') , 33 , None
('zink', 'vitamine') , 42 , 3


In [7]:
different_bigrams_weight(counters)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:03<00:00,  1.75s/it]


66876 ,  757713
('behalve', 'gp120nagalase') , 1 , None
('verdikking', 'halmleden') , 1 , None
('stobbe', 'kanttekeningen') , 1 , None
('allereerst', 'timmunocyt') , 1 , None
('bestellen', 'verspreiden') , 1 , None
('doet', 'komen') , 1 , None
('verouderingsproces', 'lijken') , 1 , None
('autoimmune', 'diseaseshave') , 1 , None
('sectie', 'getiteld') , 1 , None
('daarin', 'optreden') , 1 , None
('stikstof', 'productief') , 1 , None
('size', 'also') , 1 , None
('vrijkomt', 'burnet') , 1 , None
('mobiele', 'komen') , 1 , None
('poisoning', 'highest') , 1 , None
('statines', 'plan') , 1 , None
('moeten', 'vier') , 1 , None
('medicatieshoi', 'karenzie') , 1 , None
('included', 'various') , 1 , None
('according', 'vrain') , 1 , None
('tussen', 'beidetestgroepen') , 1 , None
('toxinenandere', 'cytochroom') , 1 , None
('boek', 'stervend') , 1 , None
('gaat', 'compenseren') , 1 , None
('geactiveerd', 'doorontwikkeld') , 1 , None
('fragmenteerbaar', 'vermoedelijk') , 1 , None
('duration', 'mont

('leafy', 'vegetables') , 1 , None
('nemen', 'samengaan') , 1 , None
('publish', 'jenner') , 1 , None
('disease', 'transmissiononly') , 1 , None
('welksubtype', 'leukemie') , 1 , None
('control', 'employees') , 1 , None
('ascribable', 'amphetamine') , 1 , None
('same', 'timesecond') , 1 , None
('local', 'healthcare') , 1 , None
('gemiddelde', 'mens') , 1 , None
('aangepastdaarnaast', 'nieuwe') , 1 , None
('huisdieren', 'honden') , 1 , None
('titel', 'opgesomde') , 1 , None
('bescherming', 'suikerbieten') , 1 , None
('middelen', 'vallen') , 1 , None
('vaccine', 'efficacy') , 1 , None
('signalen', 'aluminiumzouten') , 1 , None
('hierdoor', 'allerlei') , 1 , None
('control', 'clearance') , 2 , None
('basis', 'amfetaminen') , 1 , None
('robert', 'jacobson') , 1 , None
('juist', 'pesticiden') , 1 , None
('krijgen', 'toegediend') , 2 , None
('slaagde', 'experiment') , 1 , None
('cholesterolverlagers', 'waaronder') , 1 , None
('terug', 'verwachten') , 1 , 1
('exelproject', 'hoofdrol') , 1 , N

...

('bijvoorbeeld', 'meetapparatuur') , 1 , None
('depressie', 'amfetamine') , 1 , None
('grotendeels', 'veld') , 1 , None
('minder', 'verzadigd') , 1 , None
('cell', 'wall') , 1 , None
('effectiever', 'hartstichting') , 1 , None
('with', 'prescription') , 1 , None
('horen', 'chloorverbindingen') , 1 , None
('wandknipper', 'actie') , 1 , None
('nauwkeurig', 'vermeld') , 1 , None
('mensen', 'jonge') , 1 , None
('with', 'aluminum') , 3 , None
('2016', 'cochraneoverzicht') , 1 , None
('science', 'business') , 1 , None
('sterke', 'invloed') , 1 , None
('opbrengst', 'langere') , 1 , None
('tissues', 'related') , 1 , None
('zeer', 'reactieve') , 1 , None
('hoeveelheden', 'aluminiumzouten') , 2 , None
('april', 'resultaten') , 1 , None
('with', 'increase') , 1 , None
('wacht', 'eerst') , 1 , 1
('those', 'have') , 1 , None
('ontgiften', 'aluminiumhydroxide') , 2 , None
('concentrations', 'antibodies') , 1 , None
('work', 'babies') , 2 , None
('mensen', 'procent') , 1 , None
('bevat', 'genen') , 1

('lever', 'directe') , 1 , None
('boven', 'spandoek') , 1 , None
('this', 'artificially') , 1 , None
('when', 'infected') , 1 , None
('lichaamseigen', 'cellen') , 1 , None
('bovenstaande', 'titel') , 1 , None
('2004', 'jaar') , 1 , None
('afloopt', 'maarzink') , 1 , None
('volkskrant', 'citeer') , 1 , None
('aspirientjes', 'bijwerkingen') , 1 , None
('stuitend', 'gebrek') , 1 , None
('leerde', 'vanmorgen') , 1 , None
('tijdschrift', 'viruses') , 1 , None
('staat', 'aangegeven') , 1 , None
('been', 'advances') , 1 , None
('zodat', 'bijvoorbeeld') , 1 , 1
('komt', 'tweedosisvaccinatie') , 1 , None
('waterniveau', 'paar') , 1 , None
('bleef', 'controleglas') , 1 , None
('lage', 'spierspanning') , 1 , None
('glucose', 'balance') , 1 , None
('nagenoeg', 'onbekend') , 1 , None
('geven', 'verontrustend') , 1 , None
('kans', 'tussen') , 1 , None
('allemaal', 'goed') , 1 , 2
('extra', 'magnesium') , 1 , None
('afweer', 'werking') , 1 , None
('succesvol', 'bedwongen') , 1 , None
('voldoende', 'n

('gehaald', 'proberen') , 1 , None
('constructieve', 'oplossing') , 1 , None
('features', 'minstens') , 1 , None
('bcellen', 'enof') , 1 , None
('ontwikkelde', 'aandoeningen') , 1 , None
('observaties', 'ineens') , 1 , None
('schreef', 'origineel') , 1 , None
('zowel', 'electroactieve') , 1 , None
('krijgen', 'misschien') , 1 , 2
('laatste', 'onderzoeksbevinding') , 1 , None
('control', 'groups') , 2 , None
('geschat', '120000') , 1 , None
('ontvangen', 'kinderen') , 1 , None
('populatie', 'seizoensgriep') , 1 , None
('behoort', 'pleiten') , 1 , None
('diverse', 'technieken') , 1 , None
('this', 'picture') , 1 , None
('spierproblemen', 'innemen') , 2 , None
('gespannenik', 'aandelen') , 1 , None
('gezonde', '50plussers') , 1 , None
('through', 'clinical') , 1 , None
('oncogene', 'substantie') , 1 , None
('lang', 'omgekeerd') , 1 , None
('verzwakt', 'mazelenvirus') , 1 , None
('ripe', 'record') , 1 , None
('linie', 'hoger') , 1 , None
('oorzaken', 'jullie') , 1 , None
('grootste', 'bedr

('gemiddelde', 'mogelijk') , 1 , None
('preventieve', 'combinatiepil') , 1 , None
('aanwezigheid', 'bekend') , 1 , None
('blijft', 'lichte') , 1 , None
('eenoproep', 'vaccineren') , 1 , None
('ware', 'boodschap') , 2 , None
('geboden', 'misschien') , 1 , None
('griffin', 'pathogenesis') , 1 , None
('verder', 'gaat') , 1 , 9
('literatuur', 'blijkt') , 1 , None
('coemzyme', 'interrupted') , 1 , None
('tell', 'whether') , 1 , None
('beschikbare', 'afmetingen') , 1 , None
('risico', 'ontwikkeling') , 1 , None
('maal', 'achter') , 1 , None
('halen', 'zorgen') , 1 , None
('deur', 'email') , 1 , None
('soort', 'soort') , 1 , None
('verhoogd', 'cholesterol') , 2 , None
('intended', 'configuration') , 1 , None
('encommissievoorzitter', 'arno') , 1 , None
('vetzuurverbranding', 'mitochondrin') , 1 , None
('bovenste', 'ledematen') , 2 , None
('bevatten', 'virussen') , 1 , None
('verschillende', 'recombinantetechnieken') , 1 , None
('verf', 'inkt') , 1 , None
('goedkoop', 'patent') , 1 , None
('be

('hetformulier', 'bloedonderzoek') , 1 , None
('hiervooris', 'bepaalde') , 1 , None
('amfetamineverbindingen', 'gebruiken') , 1 , None
('glutathione', 'stransferasecytosolic') , 1 , None
('individuele', 'dokters') , 1 , None
('belangrijke', 'diagnostische') , 1 , None
('patients', 'under') , 1 , None
('oudemannelijke', 'menselijke') , 1 , None
('uitgescheiden', 'valt') , 1 , None
('werk', 'twee') , 1 , 2
('vaccinerenmet', 'antigenen') , 1 , None
('spiegels', 'verlaagt') , 1 , None
('artikelen', 'onderwerp') , 1 , None
('energiefabriekjes', 'onze') , 1 , None
('voormalig', 'minister') , 1 , 5
('expression', 'self') , 1 , None
('voedsel', 'gevonden') , 1 , None
('pharmacological', 'treatment') , 1 , None
('1899', 'jaar') , 1 , None
('gebruikt', 'legenda') , 1 , None
('achteraan', 'eigen') , 1 , None
('zinkspiegel', 'stressrespons') , 1 , None
('ronald', 'subject') , 1 , None
('ronnie', 'ssris') , 1 , None
('endoneural', 'edema') , 1 , None
('discussie', 'scherp') , 1 , None
('2862007', '